In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
from meteostat import Point, Daily

In [ ]:
df = pd.read_csv('../../data/cleaned/wildfire.csv')

df.head()

In [ ]:
df['FireDiscoveryDateTime'] = pd.to_datetime(df['FireDiscoveryDateTime'], infer_datetime_format=True, errors = 'coerce')
df['ControlDateTime'] = pd.to_datetime(df['ControlDateTime'], infer_datetime_format=True, errors = 'coerce')

In [ ]:
def get_weather_meteo(id, lat, long, start_date, end_date, result_df):
  print(id)
  loc_point = Point(lat, long)
  data = Daily(loc_point, start_date.replace(tzinfo=None), end_date.replace(tzinfo=None))
  data = data.fetch().reset_index()

  data['lat'] = lat
  data['long'] = long
  data['id'] = id
  result_df = pd.concat([result_df, data])
  return result_df

In [ ]:
meteo_res_df = pd.DataFrame()

for i in df.index:
    meteo_res_df = get_weather_meteo(
        i,
        df.loc[i, "InitialLatitude"],
        df.loc[i, "InitialLongitude"],
        df.loc[i, "FireDiscoveryDateTime"],
        df.loc[i, "ControlDateTime"],
        meteo_res_df,
    )


In [ ]:
meteo_res_df.to_csv('../../data/raw/meteo_weather.csv')